In [ ]:
import os.path
import datetime
from pathlib import Path

# essentials
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# tensorflow tools
import keras
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess
from keras.applications.xception import Xception, preprocess_input as xception_preprocess
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras import Model
from keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, AveragePooling2D, Concatenate, Add, Activation, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import plot_model

In [ ]:
base_dir = './data/Fruits-360/fruits-360_dataset/fruits-360'

train_filepaths = list(Path(base_dir + 'Training').glob(r'**/*.jpg'))
test_filepaths = list(Path(base_dir + 'Test').glob(r'**/*.jpg'))

In [ ]:
def get_fruit_label(filepaths):
    labels = [str(filepath).split('\\')[-2].split(' ')[0] for filepath in filepaths]

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    fruits_df = pd.concat([filepaths, labels], axis=1)

    return fruits_df

In [ ]:
train_df = get_fruit_label(train_filepaths)
test_df = get_fruit_label(test_filepaths)

In [ ]:
label_list = train_df.Label.unique().tolist()

In [ ]:
image_size = (224, 224, 3)
batch_size = 32
num_classes = len(label_list)

In [ ]:
train_datagen = image.ImageDataGenerator(
    validation_split=0.2,
    rotation_range=180,
    shear_range=10,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = image.ImageDataGenerator()

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    target_size=image_size[:-1]
)

validation_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    target_size=image_size[:-1]
)

test_gen = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=image_size[:-1]
)

In [ ]:
def build_model(pretrained_model):
    x = Dense(256, activation='relu')(pretrained_model.output)
    output = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=pretrained_model.input, outputs=output)

<h1> VGG16 </h1>

In [ ]:
train_datagen.preprocessing_function = vgg16_preprocess
test_datagen.preprocessing_function = vgg16_preprocess

In [ ]:
pretrained_model_vgg16 = VGG16(
    input_shape=image_size,
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model_vgg16.trainable = False

In [ ]:
model_vgg16 = build_model(pretrained_model_vgg16)
model_vgg16.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_vgg16.fit(train_gen, validation_data=validation_gen, epochs=1)

In [ ]:
loss_vgg16, accuracy_vgg16 = model_vgg16.evaluate(test_gen)

In [ ]:
test_datagen.preprocessing_function = None
X_sample, y_sample = test_gen.next()

apply_preprocess = np.vectorize(xception_preprocess)
predictions_vgg16 = model_vgg16.predict(apply_preprocess(X_sample), verbose=0)

fig, axes = plt.subplots(4, 8, figsize=(15, 7), subplot_kw={'xticks' : [], 'yticks' : []})
for i, ax in enumerate(axes.flat):
    ax.imshow(X_sample[i]/255.)
    ax.set_title(f'True: {label_list[np.argmax(y_sample[i])]}\nPredicted: {label_list[np.argmax(predictions_vgg16[i])]}', fontsize=8)
plt.tight_layout(pad=0.5)
plt.show()

<h1> Xception </h1>

In [ ]:
train_datagen.preprocessing_function = xception_preprocess
test_datagen.preprocessing_function = xception_preprocess

In [ ]:
pretrained_model_xception = Xception(
    input_shape=image_size,
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model_xception.trainable = False

In [ ]:
model_xception = build_model(pretrained_model_xception)
model_xception.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_xception.fit(train_gen, validation_data=validation_gen, batch_size=batch_size, epochs=1)

In [ ]:
loss_xception, accuracy_xception = model_xception.evaluate(test_gen)

In [ ]:
test_datagen.preprocessing_function = None
X_sample, y_sample = test_gen.next()

apply_preprocess = np.vectorize(xception_preprocess)
predictions_xception = model_xception.predict(apply_preprocess(X_sample), verbose=0)

fig, axes = plt.subplots(4, 8, figsize=(15, 7), subplot_kw={'xticks' : [], 'yticks' : []})
for i, ax in enumerate(axes.flat):
    ax.imshow(X_sample[i]/255.)
    ax.set_title(f'True: {label_list[np.argmax(y_sample[i])]}\nPredicted: {label_list[np.argmax(predictions_xception[i])]}', fontsize=8)
plt.tight_layout(pad=0.5)
plt.show()